In [ ]:
#creating function to pull ticker names
def training_model(Current_ticker):
    df_ibm_ticker = api.get_barset(
       tickers,
       timeframe,
       start=start_date,
       end=end_date,
       limit=1000
    ).df
    # Display sample data
    df_ibm_ticker.head()
    
    #
    data = np.array(df_ibm_ticker)

    # train test split, we can take last 100 data points as test set
    train , test = data[0:-100], data[-100:]
    
   
        # Predict Closing Prices using a 10 day window of previous closing prices
    # Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
    window_size = 10

    # Column index 0 is the 'IBM ' column
    # Column index 1 is the `Close` column
    feature_column = 1
    target_column = 1
    X, y = window_data(df_ibm_ticker, window_size, feature_column, target_column)
    

        # Use 70% of the data for training and the remaineder for testing
    data_train = int(.7 * len(X))
    X_train = X[:data_train - 1]
    X_test = X[data_train:]
    y_train = y[:data_train - 1]
    y_test = y[data_train:]
    
        # Use the MinMaxScaler to scale data between 0 and 1.
    from sklearn.preprocessing import MinMaxScaler
    x_train_scaler = MinMaxScaler()
    x_test_scaler = MinMaxScaler()
    y_train_scaler = MinMaxScaler()
    y_test_scaler = MinMaxScaler()

    # Fit the scaler for the training Data
    x_train_scaler.fit(X_train)
    y_train_scaler.fit(y_train)

    # Scale the training data
    X_train = x_train_scaler.transform(X_train)
    y_train = y_train_scaler.transform(y_train)

    # Fit the scaler for the testing Data
    x_test_scaler.fit(X_test)
    y_test_scaler.fit(y_test)

    # Scale the y_test data
    X_test = x_test_scaler.transform(X_test)
    y_test = y_test_scaler.transform(y_test)

        # Reshape the features for the model
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model. 
        # Note: The input shape is the number of time steps and the number of indicators
    # Note: Batching inputs has a different input shape of Samples/TimeSteps/Features
    model = Sequential()
    model.add(LSTM(
        units=30, return_sequences=True,
        input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=30))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
        # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

        # Summarize the model
    model.summary()
    
        # Train the model
    # Use at least 10 epochs
    model.fit(X_train, y_train, epochs=100, shuffle=False, batch_size=1,
        verbose=1)
    
            # Make some predictions
        predicted = model.predict(X_test)

        # Recover the original prices instead of the scaled version
    predicted_prices = y_test_scaler.inverse_transform(predicted)
    real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))
    
        # Create a DataFrame of Real and Predicted values
    stocks = pd.DataFrame({
        "Real": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = df_ibm_ticker.index[-len(real_prices): ]) 
    stocks.head()
    
    return stocks